In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# logistic regression
from sklearn.linear_model import LogisticRegression

In [2]:
# read in data
df = pd.read_csv('cleaned_data.csv')

# list all column names without ...
pd.set_option('display.max_columns', None)
df.head()

C:\Users\Gian\AppData\Local\Temp\ipykernel_17532\689264397.py:2: DtypeWarning: Columns (21,22,36,37,46,47,48,49,50,51,52,53,54,56,110,111,112,117) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('cleaned_data.csv')


,Reporting Railroad Name,Report Year,Accident Number,Accident Year,Accident Month,Other Railroad Name,Other Accident Number,Other Accident Year,Other Accident Month,Maintenance Railroad Name,Maintenance Accident Number,Maintenance Accident Year,Maintenance Accident Month,Grade Crossing ID,Day,Date,Time,Accident Type,Hazmat Cars,Hazmat Cars Damaged,Hazmat Released Cars,Persons Evacuated,Subdivision,Division Code,Division,Station,Milepost,State Abbreviation,State Name,County Name,District,Temperature,Visibility,Weather Condition,Track Type,Track Name,Track Class,Track Density,Train Direction,Equipment Type,Equipment Attended,Train Number,Train Speed,Recorded Estimated Speed,Maximum Speed,Gross Tonnage,Signalization,Method of Operation,Adjunct Code 1,Adjunct Code Name 1,Adjunct Code 2,Adjunct Code Name 2,Adjunct Code 3,Adjunct Code Name 3,Remote Control Locomotive,First Car Initials,First Car Number,First Car Position,First Car Loaded,Causing Car Initials,Causing Car Number,Causing Car Position,Causing Car Loaded,Positive Alcohol Tests,Positive Drug Tests,Passengers Transported,Head End Locomotives,Mid Train Manual Locomotives,Mid Train Remote Locomotives,Rear End Manual Locomotives,Rear End Remote Locomotives,Derailed Head End Locomotives,Derailed Mid Train Manual Locomotives,Derailed Mid Train Remote Locomotives,Derailed Rear End Manual Locomotives,Derailed Rear End Remote Locomotives,Loaded Freight Cars,Loaded Passenger Cars,Empty Freight Cars,Empty Passenger Cars,Cabooses,Derailed Loaded Freight Cars,Derailed Loaded Passenger Cars,Derailed Empty Freight Cars,Derailed Empty Passenger Cars,Derailed Cabooses,Equipment Damage Cost,Track Damage Cost,Total Damage Cost,Primary Accident Cause,Contributing Accident Cause,Engineers On Duty,Firemen On Duty,Conductors On Duty,Brakemen On Duty,Hours Engineers On Duty,Minutes Engineers On Duty,Hours Conductors On Duty,Minutes Conductors On Duty,Railroad Employees Killed,Railroad Employees Injured,Passengers Killed,Passengers Injured,Others Killed,Others Injured,Persons Killed For Reporting Railroad,Persons Injured For Reporting Railroad,Total Persons Killed,Total Persons Injured,Total Killed Form 54,Total Injured Form 54,Special Study 1,Special Study 2,Latitude,Longitude,Narrative,Joint Track Type,Joint Track Class,Class,Joint CD,Incident Key,Report Key,Reporting Railroad Company Grouping,Reporting Railroad Class,Reporting Railroad SMT Grouping,Reporting Parent Railroad Company Name,Reporting Railroad Holding Company,Other Railroad Company Grouping,Other Railroad Class,Other Railroad SMT Grouping,Other Parent Railroad Company Name,Other Railroad Holding Company,Maintenance Railroad Company Grouping,Maintenance Railroad Class,Maintenance Railroad SMT Grouping,Maintenance Parent Railroad Company Name,Maintenance Railroad Holding Company
0,Northern Indiana Commuter Transportation District,2017.0,N1700013,17.0,6.0,NaN,NaN,NaN,NaN,NORTHEAST ILLINOIS REGIONAL COMMUTER RAIL CORP...,201707188,17.0,6.0,NaN,6.0,06/06/2017,2:14 PM,Derailment,0.0,0.0,0.0,149,SYSTEM,NaN,NaN,CHICAGO,.55,IL,ILLINOIS,COOK,4.0,65.0,Day,Clear,Yard,YARD LEAD,1,NaN,North,Commuter Train - Pulling,Yes,918,10.0,Estimated,10,0,Not Signaled,Other Than Main Track,NaN,NaN,NaN,NaN,NaN,NaN,Not a remotely controlled operation,NICD,000203,2.0,No,NaN,NaN,0.0,NaN,0.0,0.0,Yes,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,"98,627",0,"132,013",Switch point worn or broken,NaN,1.0,NaN,1.0,NaN,1.0,56.0,1.0,56.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,OTH,000-000-000,41.884035,-87.623003,TRAIN 918 (NICD WB TRAIN 18) DERAILED WHILE PA...,Yard,1,No,1.0,NIRC201707188201706,NICDN1700013201706,Passenger (Formerly Commuter),Class 3,SMT-4 - CP/CN/CCD,Northern Indiana Commuter Transportation District,Not Assigned,NaN,NaN,NaN,NaN,NaN,Passenger (Formerly Commuter),Class 3,SMT-4 - CP/CN/CCD,NORTHEAST ILLINOIS REGIONAL COMMUTER RAIL CORP...,Not Assigned
1,NORTHEAST ILLINOIS REGIONAL COMMUTER RAIL CORP...,2017.0,201707

In [3]:
# get all Accident Types
df['Accident Type'].unique()

array(['Derailment', 'Side collision', 'Hwy-rail crossing', 'Obstruction',
       'Other (describe in narrative)', 'Raking collision',
       'Other impacts', 'Fire/violent rupture', 'Head on collision',
       'Rear end collision', 'Broken train collision',
       'RR grade crossing', nan, 'Explosion-detonation'], dtype=object)

In [4]:
# # array(['Derailment', 'Side collision', 'Hwy-rail crossing', 'Obstruction',
# #        'Other (describe in narrative)', 'Raking collision',
# #        'Other impacts', 'Fire/violent rupture', 'Head on collision',
# #        'Rear end collision', 'Broken train collision',
# #        'RR grade crossing', nan, 'Explosion-detonation'], dtype=object)
# # Give a severity score to each accident type

# # Remove other impacts and other (describe in narrative) because they are too vague
# df = df[df['Accident Type'] != 'Other impacts']
# df = df[df['Accident Type'] != 'Other (describe in narrative)']

# severity = {'Derailment': 5, 'Side collision': 4, 'Hwy-rail crossing': 2, 'Obstruction': 4,
#             'Raking collision': 3, 'Fire/violent rupture': 3, 'Head on collision': 3,
#             'Rear end collision': 3, 'Broken train collision': 3, 'RR grade crossing': 3,
#             'Explosion-detonation': 2}

In [5]:
# Want to see if certain weather conditions are more likely to cause certain accidents types
accident_types = df['Accident Type'].unique()
weather_types = df['Weather Condition'].unique()
weather_types

array(['Clear', 'Snow', 'Cloudy', 'Rain', 'Fog', 'Sleet', nan],
      dtype=object)

In [6]:
# Weather Condition, Temperature, Visibility, Accident Month
# use these to predict accident type
# support vector machine
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Give each row an id
df['id'] = df.index

# preprocessing data
X = df[['id', 'Weather Condition', 'Temperature', 'Visibility', 'Accident Month']]
# Convert the categorical data into numerical data
X = pd.get_dummies(X)
# Drop the column with NaN values
X = X.dropna(axis=1)

y = df[['id', 'Accident Type']]
# drop rows with NaN values
y = y.dropna(axis=0)

# keep only rows that have a matching id in both X and y
X = X[X['id'].isin(y.index)]
y = y[y.index.isin(X['id'])]

# Drop id column
X = X.drop('id', axis=1)
y = y.drop('id', axis=1)

# split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# train model
model = SVC(verbose=1)
model.fit(X_train, y_train)

# test model
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

c:\ProgramData\miniconda3\envs\treten\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[LibSVM]